# Part 3: Unbiased Evaluation using a New Test Set

In this part, we are given a new test set (`/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv`). We can now take advantage of the entire smart sample that we created in Part I. 

* Retrain a pipeline using the optimal parameters that the pipeline learned. We don't need to repeat GridSearch here. 

## Import modules as needed

In [38]:
%matplotlib inline
import matplotlib.pyplot as plt
import joblib
import os, sys
import itertools
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

## Load smart sample and the best pipeline from Part II

In [39]:
train_set_downsampled= joblib.load('train-set-downsized')
train_set_downsampled = train_set_downsampled.dropna()

best_pipe = joblib.load('bestpiperevised')
best_pipe

Pipeline(steps=[('FactorAnalysis', FactorAnalysis(n_components=15)),
                ('RandomForestClassifier',
                 RandomForestClassifier(n_estimators=20))])


##  Retrain a pipeline using the full sampled training data set

Use the full sampled training data set to train the pipeline.

In [56]:
# Add code below this comment  (Question #E301)
# ----------------------------------

X = train_set_downsampled.iloc[:,:-1]
y = train_set_downsampled.went_on_backorder


best_pipe.fit(X,y)



Pipeline(steps=[('FactorAnalysis', FactorAnalysis(n_components=15)),
                ('RandomForestClassifier',
                 RandomForestClassifier(n_estimators=20))])

In [57]:
lof_labels = LocalOutlierFactor(n_neighbors=10).fit_predict(X, y)
inliers = lof_labels == 1
X_clean = X[inliers]
y_clean = y[inliers]

MemoryError: Unable to allocate 1.00 GiB for an array with shape (6214, 21596) and data type float64

### Save the trained model with the pickle library.

In [15]:
# Add code below this comment  
# -----------------------------
joblib.dump(best_pipe,'bestpipefittedwithfullsampledtrainingset')





['bestpipefittedwithfullsampledtrainingset']


## Load the Testing Data and evaluate your model

 * `/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv`
 
* We need to preprocess this test data (follow the steps similar to Part I)
* If we have fitted any normalizer/standardizer in Part 2, then we have to transform this test data using the fitted normalizer/standardizer

In [46]:
# Preprocess the given test set  (Question #E302)
# ----------------------------------

testdataset = '/dsa/data/all_datasets/back_order/Kaggle_Test_Dataset_v2.csv'
assert os.path.exists(testdataset)

testdata = pd.read_csv(testdataset).sample(frac = 1).reset_index(drop=True)


testdata = testdata.drop(columns=['sku'])

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
testdata.head()
yes_no_columns = list(filter(lambda i: testdata[i].dtype!=np.float64, testdata.columns))
print(yes_no_columns)


for column_name in yes_no_columns:
    mode = testdata[column_name].apply(str).mode()[0]
    print('Filling missing values of {} with {}'.format(column_name, mode))
    testdata[column_name].fillna(mode, inplace=True)

['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']
Filling missing values of potential_issue with No
Filling missing values of deck_risk with No
Filling missing values of oe_constraint with No
Filling missing values of ppap_risk with No
Filling missing values of stop_auto_buy with Yes
Filling missing values of rev_stop with No
Filling missing values of went_on_backorder with No


In [48]:
encoder = LabelEncoder()
testdata['potential_issue'] = encoder.fit_transform(testdata['potential_issue'])

testdata['deck_risk'] = encoder.fit_transform(testdata['deck_risk'])

testdata['oe_constraint'] = encoder.fit_transform(testdata['oe_constraint'])

testdata['ppap_risk'] = encoder.fit_transform(testdata['ppap_risk'])

testdata['stop_auto_buy'] = encoder.fit_transform(testdata['stop_auto_buy'])

testdata['rev_stop'] = encoder.fit_transform(testdata['rev_stop'])

testdata['went_on_backorder'] = encoder.fit_transform(testdata['went_on_backorder'])

In [49]:
testdata.head()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,80.0,2.0,2.0,45.0,90.0,135.0,14.0,44.0,95.0,136.0,...,0.0,0.48,0.36,0.0,0,0,0,1,0,0
1,67.0,8.0,6.0,38.0,104.0,183.0,14.0,59.0,134.0,199.0,...,0.0,0.85,0.84,0.0,0,0,0,1,0,0
2,18.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.73,0.78,0.0,0,0,0,1,0,0
3,2.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.77,0.73,0.0,0,0,1,1,0,0
4,4.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.95,0.97,0.0,0,0,0,1,0,0


In [44]:
testdata = testdata.fillna(testdata.mean())

We can now predict and evaluate with the preprocessed test set. It would be interesting to see the performance with and without outliers removal from the test set. We can report confusion matrix, precision, recall, f1-score, accuracy, and other measures (if any). 

In [45]:
# Add code below this comment  (Question #E303)
# ----------------------------------

Xtest = testdata.iloc[:,:-1]
ytest = testdata.went_on_backorder


y_pred = best_pipe.predict(Xtest)


print(pd.DataFrame(confusion_matrix(ytest, y_pred)))
print("Accuracy:", np.round(accuracy_score(ytest, y_pred), 2))
print("Precision:", np.round(precision_score(ytest, y_pred, average='weighted'), 2))
print("Recall:", np.round(precision_score(ytest, y_pred, average='weighted'), 2))
print("F1-Score:", np.round(f1_score(ytest, y_pred, average='weighted'), 2))






        0      1
0  205965  33423
1     542   2146
Accuracy: 0.86
Precision: 0.99
Recall: 0.99
F1-Score: 0.91


## Conclusion

## Reflect

Imagine you are data scientist that has been tasked with developing a system to save your 
company money by predicting and preventing back orders of parts in the supply chain.

Write a **brief summary** for "management" that details your findings, 
your level of certainty and trust in the models, 
and recommendations for operationalizing these models for the business.

# Save your notebook!
## Then `File > Close and Halt`